In [4]:
from enum import Enum
from random import randint, choice
from copy import copy

E, X, O = ' ', 'X', 'O'

class GameEnvironment():
    def __init__(self, initial_state=None):
        if initial_state == None:
            self.__initial_state = [E for n in range(9)]  # start with empty board
        else:
            self.__initial_state = copy(initial_state)  # copy to prevent aliassing
        self.__state = self.__initial_state
        self.__possible_states = []
        self.__calculate_possible_states(self.__initial_state)
        
    def __calculate_possible_states(self, state):
        actions = self.get_possible_actions(state)
        for action in actions:
            new_state = copy(state)
            if state.count(X) == state.count(O):
                new_state[action] = X 
            else: 
                new_state[action] = O
            self.__possible_states.append(new_state)
            if not self.is_done(new_state):
                self.__calculate_possible_states(new_state)

    def reset(self):
        self.__state = self.__initial_state
        return self.__state
    
    def __calculate_transition(self, action):
        # change the state to reflect the move by the agent
        self.__state[action] = X
        if self.is_done():
            return self.__state
        # let the opponent make a random move
        opponent_action = choice(self.get_possible_actions())
        self.__state[opponent_action] = O
        return self.__state
      
    def step(self, action):
        old_state = self.__state
        self.__state = self.__calculate_transition(action)  # state after action
        observation = self.__state  # environment is fully observable
        done = self.is_done()
        reward = self.get_reward(self.__state)
        info = {}  # optional debug info
        return observation, done, reward, info

    def render(self):
        BACKGROUND = [
            '   │   │   ',
            '───┼───┼───',
            '   │   │   ',
            '───┼───┼───',
            '   │   │   '
        ]
        rendering = copy(BACKGROUND)
        for n, S_n in enumerate(self.__state):
            if S_n != E:
                row = 2 * (n // 3)
                col = 4 * (n % 3) + 1
                line = rendering[row]
                rendering[row] = line[:col] + S_n + line[col + 1:]
        
        for line in rendering:
            print(line)
        
    #=========================================================
    # public functions for agent to calculate optimal policy
    #=========================================================
    
    def get_possible_states(self):
        return self.__possible_states
    
    def get_possible_actions(self, state=None):
        if state is None:
            state = self.__state
        return [n for n in range(9) if state[n] == E]

    def is_done(self, state=None):
        if state is None:
            state = self.__state
        if E not in state:
            return True
        if state[0] == state[1] == state[2] != E:
            return True
        if state[3] == state[4] == state[5] != E:
            return True
        if state[6] == state[7] == state[8] != E:
            return True
        if state[0] == state[3] == state[6] != E:
            return True
        if state[1] == state[4] == state[7] != E:
            return True
        if state[2] == state[5] == state[8] != E:
            return True
        if state[0] == state[4] == state[8] != E:
            return True
        if state[2] == state[4] == state[6] != E:
            return True
        return False
    
    def get_reward(self, state):
        # Reward R(s) for every possible state
        if state[0] == state[1] == state[2] != E:
            return 1 if state[0] == X else -1
        if state[3] == state[4] == state[5] != E:
            return 1 if state[0] == X else -1
        if state[6] == state[7] == state[8] != E:
            return 1 if state[0] == X else -1
        if state[0] == state[3] == state[6] != E:
            return 1 if state[0] == X else -1
        if state[1] == state[4] == state[7] != E:
            return 1 if state[0] == X else -1
        if state[2] == state[5] == state[8] != E:
            return 1 if state[0] == X else -1
        if state[0] == state[4] == state[8] != E:
            return 1 if state[0] == X else -1
        if state[2] == state[4] == state[6] != E:
            return 1 if state[0] == X else -1
        return False
        
    def get_transition_prob(self, action, new_state, old_state=None):
        if old_state is None:
            old_state = self.__state
        # returns the Transition Probability P(s'| s, a)
        # with s = old_state, a = action and s' = new_state

        # if the game is over, no transition can take place
        if self.is_done(old_state):
            return 0.0
        
        # the position of the action must be empty
        if old_state[action] != E:
            return 0.0
        
        # state after placing X
        state_after_X = copy(old_state)  # avoid unwanted changed by reference
        state_after_X[action] = X

        # check if game is done
        if self.is_done(state_after_X) and state_after_X == new_state:
            return 1.0

        # game is not done: calculate all possible states of the opponent
        possible_new_states = []
        possible_opponent_actions = self.get_possible_actions(state_after_X)
        for action in possible_opponent_actions:
            possible_new_state = copy(state_after_X)
            possible_new_state[action] = O
            possible_new_states.append(possible_new_state)
        if new_state not in possible_new_states:
            return 0.0
        
        # transition is possible, apply strategy:
        # random opponent, probability is 1 / (# of E before placing the new O)
        prob = 1 / (len(possible_new_states))
        return prob

In [5]:
# example of creation of an environment in the default state
mdp = GameEnvironment()
mdp.reset()
mdp.render()
state, done, reward, info = mdp.step(4)
print('state =', state, ', reward =', reward, ', done =', done)
mdp.render()
print('possible (internal) game states:')
mdp.get_possible_states()

   │   │   
───┼───┼───
   │   │   
───┼───┼───
   │   │   
state = [' ', ' ', ' ', ' ', 'X', ' ', ' ', 'O', ' '] , reward = False , done = False
   │   │   
───┼───┼───
   │ X │   
───┼───┼───
   │ O │   
possible (internal) game states:


[['X', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['X', 'O', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['X', 'O', 'X', ' ', ' ', ' ', ' ', ' ', ' '],
 ['X', 'O', 'X', 'O', ' ', ' ', ' ', ' ', ' '],
 ['X', 'O', 'X', 'O', 'X', ' ', ' ', ' ', ' '],
 ['X', 'O', 'X', 'O', 'X', 'O', ' ', ' ', ' '],
 ['X', 'O', 'X', 'O', 'X', 'O', 'X', ' ', ' '],
 ['X', 'O', 'X', 'O', 'X', 'O', ' ', 'X', ' '],
 ['X', 'O', 'X', 'O', 'X', 'O', 'O', 'X', ' '],
 ['X', 'O', 'X', 'O', 'X', 'O', 'O', 'X', 'X'],
 ['X', 'O', 'X', 'O', 'X', 'O', ' ', 'X', 'O'],
 ['X', 'O', 'X', 'O', 'X', 'O', 'X', 'X', 'O'],
 ['X', 'O', 'X', 'O', 'X', 'O', ' ', ' ', 'X'],
 ['X', 'O', 'X', 'O', 'X', ' ', 'O', ' ', ' '],
 ['X', 'O', 'X', 'O', 'X', 'X', 'O', ' ', ' '],
 ['X', 'O', 'X', 'O', 'X', 'X', 'O', 'O', ' '],
 ['X', 'O', 'X', 'O', 'X', 'X', 'O', 'O', 'X'],
 ['X', 'O', 'X', 'O', 'X', 'X', 'O', ' ', 'O'],
 ['X', 'O', 'X', 'O', 'X', 'X', 'O', 'X', 'O'],
 ['X', 'O', 'X', 'O', 'X', ' ', 'O', 'X', ' '],
 ['X', 'O', 'X', 'O', 'X', 'O', 'O', 'X'

In [ ]:
mdp = TicTacToeMDPEnvironment(['X', ' ', ' ', 
                               'O', 'X', ' ', 
                               ' ', ' ', 'O'])
mdp.render()
possible_actions = mdp.get_possible_actions()
print('possible actions: ', possible_actions)
random_agent_action = choice(possible_actions)
new_state, done, reward, info = mdp.step(random_agent_action)
mdp.render()
possible_actions = mdp.get_possible_actions()
print('possible actions: ', possible_actions)

In [ ]:
S_0 = ['X', ' ', ' ', 
       'O', 'X', ' ', 
       ' ', ' ', 'O']

S_1 = ['X', 'X', 'O', 
       'O', 'X', ' ', 
       ' ', ' ', 'O']

S_2 = ['X', 'X', ' ', 
       'O', 'X', 'O', 
       ' ', ' ', 'O']

S_3 = ['X', 'X', ' ', 
       'O', 'X', ' ', 
       'O', ' ', 'O']

S_4 = ['X', 'X', ' ', 
       'O', 'X', ' ', 
       ' ', 'O', 'O']

S_5 = ['X', 'X', ' ', 
       'O', 'X', ' ', 
       'X', ' ', 'O']

mdp = TicTacToeMDPEnvironment(S_0)
mdp.render()

print('possible actions:', mdp.get_possible_actions())

for n, S_p in enumerate([S_1, S_2, S_3, S_4, S_5], 1):
    print('S_0 -> action 1 -> S_' + str(n), 'has probability:', mdp.get_transition_prob(1, new_state=S_p))